In [1]:
from azure.cognitiveservices.language.luis.authoring import LUISAuthoringClient
from azure.cognitiveservices.language.luis.authoring.models import ApplicationCreateObject, ExampleLabelObject
from azure.cognitiveservices.language.luis.runtime import LUISRuntimeClient
from msrest.authentication import CognitiveServicesCredentials
from functools import reduce

import json, time, uuid

import keys

In [5]:
authoringKey = keys.authoringKey
authoringEndpoint = keys.authoringEndpoint
predictionKey = keys.predictionKey
predictionEndpoint = keys.predictionEndpoint
app_id = keys.app_id

In [3]:
client = LUISAuthoringClient(authoringEndpoint, CognitiveServicesCredentials(authoringKey))

In [46]:
app_id = client.apps.add({
            'name': "FlyMe",
            'initial_version_id':  "0.1",
            'description': "Flight booking chatbot app",
            'culture': 'en-us',
        })

print("Created app.")

Created app.


In [6]:
luis_app = client.apps.get(app_id=app_id)

app_version = luis_app.active_version

In [49]:
entities = ['or_city', 'dst_city', 'str_date', 'end_date', 'budget']

In [50]:
# Create intent into the model
intent_name = "BookFlight"

In [51]:
# Create entities and entity features into the model
for entity in entities:
    client.model.add_entity(app_id, "0.1", entity)
    print("Added entity {}".format(entity))

Added entity or_city
Added entity dst_city
Added entity str_date
Added entity end_date
Added entity budget


In [52]:
client.model.add_intent(app_id, "0.1", "BookFlight")
print("Added intent BookFlight")

Added intent BookFlight


In [66]:
with open("train.json") as f:
    train_data = json.load(f)

# send the training data to the service in chunks of 100

for i in range(0, len(train_data), 100):
    j = i + 100
    if j > len(train_data):
        j = len(train_data)

        client.examples.batch(
                    app_id,
                    app_version,
                    train_data[i:j]
                )

In [67]:
async_training = client.train.train_version(app_id, app_version)
is_trained = async_training.status == "UpToDate"

trained_status = ["UpToDate", "Success"]
while not is_trained:
    time.sleep(1)
    status = client.train.get_status(app_id, app_version)
    is_trained = all(
        m.details.status in trained_status for m in status)

print("Your app is trained. You can now go to the LUIS portal and test it!")

Your app is trained. You can now go to the LUIS portal and test it!


In [ ]:
publish_result = client.apps.publish(
    app_id,
    {
        'version_id': app_version,
        'is_staging': False,
        'region': 'westeurope'
    }
)

In [15]:
runtime_client = LUISRuntimeClient(predictionEndpoint, CognitiveServicesCredentials(predictionKey))

query = "I want to book a flight from Strasbourg to London from 10/10/2019 to 15/10/2019 for 200 euros"
result = runtime_client.prediction.resolve(app_id, query)

print("\nDetected intent: {} (score: {:d}%)".format(
            result.top_scoring_intent.intent,
            int(result.top_scoring_intent.score*100)
        ))
print("Detected entities:")
for entity in result.entities:
    print("\t-> Entity '{}' (type: {}, score:{:d}%)".format(
        entity.entity,
        entity.type,
        int(entity.additional_properties['score']*100)
    ))
print("\nComplete result object as dictionnary")
print(result.as_dict())


Detected intent: BookFlight (score: 99%)
Detected entities:
	-> Entity '200 euros' (type: budget, score:69%)
	-> Entity 'london' (type: dst_city, score:93%)
	-> Entity 'strasbourg' (type: or_city, score:99%)

Complete result object as dictionnary
{'query': 'I want to book a flight from Strasbourg to London from 10/10/2019 to 15/10/2019 for 200 euros', 'top_scoring_intent': {'intent': 'BookFlight', 'score': 0.99955165}, 'entities': [{'score': 0.69389737, 'entity': '200 euros', 'type': 'budget', 'start_index': 84, 'end_index': 92}, {'score': 0.9351498, 'entity': 'london', 'type': 'dst_city', 'start_index': 43, 'end_index': 48}, {'score': 0.9905306, 'entity': 'strasbourg', 'type': 'or_city', 'start_index': 29, 'end_index': 38}]}


## Evaluating the model

In [16]:
with open("test.json") as f:
    test_data = json.load(f)

In [40]:
test_data[2]

{'text': 'hi there! got anything from kabul to tel aviv?',
 'intentName': 'BookFlight',
 'entityLabels': [{'entityName': 'or_city',
   'startCharIndex': 28,
   'endCharIndex': 33},
  {'entityName': 'dst_city', 'startCharIndex': 37, 'endCharIndex': 45}]}

In [46]:
result = runtime_client.prediction.resolve(app_id, test_data[2]["text"])
dict_result = result.as_dict()
dict_result

{'query': 'hi there! got anything from kabul to tel aviv?',
 'top_scoring_intent': {'intent': 'BookFlight', 'score': 0.9975568},
 'entities': [{'score': 0.7219019,
   'entity': 'tel aviv ?',
   'type': 'dst_city',
   'start_index': 37,
   'end_index': 45},
  {'score': 0.99454474,
   'entity': 'kabul',
   'type': 'or_city',
   'start_index': 28,
   'end_index': 32}]}

In [62]:
keys = list(dict_result['entities'][0].keys())

In [63]:
keys

['score', 'entity', 'type', 'start_index', 'end_index']